# Satellite Remote Sensing
## From an exercise by Zac Hatfield-Dodds

We're going to look at satellite images that tell us about the environment.  
More specifically, we're going to look at some [vegetation indices](https://en.wikipedia.org/wiki/Vegetation_Index) 
Understanding vegetation properties and and how they change over time is especially important in Australia, where most ecosystems are prone to both drought and fire.

Our goals for today:

1. We'll start by opening and viewing an image of ANU, and check that the data makes sense.
2. Then, I'll give a worked example of calculating [NDVI, the normalised difference vegetation index](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index), which we can use to investigate the ANU playing fields.
3. After that, you'll look at a time-series of images from before, during, and after one of the Black Saturday bushfires.
   Analysing vegetation change can help us understand what happened in this location.
   
## Part 1: loading the ANU data

In [ ]:
# As usual, we start with our standard imports
import numpy as np
import pandas as pd
import matplotlib as plt

# And for remote sensing, we add Xarray - it's inspired by Pandas, and designed
# to handle groups of multi-dimensional arrays.  See http://xarray.pydata.org/
import xarray as xr

# Finally, we'll use a "magic" - that's the official name - to show plots inline.
%matplotlib inline
# (starting with `%` is invalid syntax in Python, so IPython and Jupyter use
#  it to denote special commands that change e.g. how they render plots for us)

In [ ]:
# We'll start by looking at an aerial photo of ANU, taken in 2014.
# This is just a few bands from a hyperspectral survey.
da = xr.open_dataarray("anu_array.nc")
# Conventional name for a DataArray is "da", to match pd.DataFrame -> "df" and xr.DataSet -> "ds".
# Interesting aspects of this array: the band dimension has coordinates, but x and y don't!
da

In [ ]:
# Lets see what the default plot looks like:
da.plot()

This is the default plot type for multi-dimensional arrays of data: a [histogram](https://en.wikipedia.org/wiki/Histogram) of value counts.  We can see that dark pixels are most common, but don't know anything about the spatial or spectral structure of the image.  If I didn't know, I'd bet that this isn't an image of snow, ice, or a salt pan - all of which are bright white - but beyond that it's hard to tell.

To make other kind of plots, we need to select a subset of the data, and plot that.  We'll get a line plot for 1D data, or an image for 2D.

(you can read [the Xarray plotting docs](http://xarray.pydata.org/en/stable/plotting.html) for more options, but we'll stick to the defaults for now)

In [ ]:
# We can select a single step in the "band" dimension by name using the `.sel` method, because it has coordinates.
# For x and y, we would need to use `.isel` with a normal index.
# Try adding `.isel(x=220)` before the `.plot()` call - what does this show?
da.sel(band="nir").plot(figsize=(8,8))

Now, let's select particular areas of the dataset in `(x, y)` space.  
We can plot these as images again, or draw another histogram!

I've drawn one for the lake (top-left) below - try selecting one of the playing fields or built-up areas.

In [ ]:
# A `slice` describes a selection with a start, stop, and step value. 
# See https://docs.python.org/3/library/functions.html#slice
lake = da.isel(x=slice(420, 800), y=slice(0, 300))
# ...and plot it as a histogram.
lake.plot()

In [ ]:
# If we plot it as an image, note the x- and y-tick labels.  No coordinates!
# The scale of the colorbar is also different; more on this in a minute.
lake.sel(band="nir").plot(figsize=(12,12))

Now, let's use the `col` and `col_wrap` argument to make a faceted plot, where we can see all the bands:

In [ ]:
# Here, we use the `robust=True` argument to automatically set the extent 
# of the colormap between the 2nd and 98th percentiles of the data.
# You can use `vmin` and `vmax` to set exact limits as well or instead.
da.plot(col="band", col_wrap=2, vmin=0,robust=True,figsize=(12,12),cmap=plt.cm.summer)

So ANU seems much brighter in the near-infrared band than in visible light! (is this unexpected, or reasonable?)

However, clipping the range of values like this means we can't tell if the bright spot (around x=380, y=180) in our image of the blue channel is also present in other bands. Start by checking google maps, and form a hypothesis. What do you think this might be, and do you expect it to be bright or dark in other bands?


In [ ]:
da.plot(col="band", col_wrap=2, cmap=plt.cm.seismic)

What.  There's a bright dot in all the bands, but the whole display is different!

Xarray has given us a *diverging* colormap, which means there must be some negative numbers in this dataset.  But each number is - or should be - an amount of light that reached the sensor, and therefore negative values should be impossible.

How could this have happened?  

Once you're done let's investigate the "weird pixels" quickly.

In [ ]:
# Taking the minimum along the x and y dimensions leaves us with a 
# one-dimensional array - the minimum value for each band. 
da.min(dim=("x", "y"))

In [ ]:
# Hmm.  We'll try selecting only the negative numbers, 
da.where(da < 0).sel(band="blue").plot()

There are only a few, so it doesn't look too important - another case for human judgement.  We have two choices here:

- just clip negative numbers to zero and move on with our lives (expedient), or
- replace negative numbers with "not a number" and put up with the gaps (principled but inconvenient) 

In [ ]:
da = da.clip(min=0)
# or, `da = da.where(da >= 0)`

# Ahh, that's much better.
da.plot(col="band", col_wrap=2)

## Part 2: a vegetation index

We've loaded and displayed our data, and we're *reasonably* confident that it's physically meaningful.  We do need to bear in mind that this is *uncorrected brightness* - that is, we know that the sensor recorded a brightness value (close to true brightness, we hope) but do not know the relationship between "amount of light" and "proportion of light diffusely reflected" for each pixel.

The way the data was collected and pre-processed determines what analyses will be meaningful.  Unfortunately there's no way for Python to stop us from precisely calculating a meaningless result; that's still a job for thoughtful humans.

Let's start by investigating ANU's playing fields, as they give us relatively large areas of a single vegetation types to work with.  For this trick, we're on the fact that a ratio index like NDVI is useful with pretty dodgy data, because it's designed to make the errors (from e.g. slope, shadow, solar angle, image angle) cancel out between bands as much as possible.

We'll start by rendering a true-color image for comparison.  Zac Hatfield-Dodds wrote this code because true- or false-color RGB images are often useful, but used to be really really annoying to draw; now it's just cryptic.

In [ ]:
# You do not need to and are not expected to know how this works, just that it does.
dat = da.transpose()
#da.sel(band=["red", "green", "blue"]).plot.imshow(robust=True)
dat.sel(band=["nir", "red", "green"]).plot.imshow(robust=True,figsize=(14.8,16))

Can you count the playing fields in this image?  Compare your count and locations with someone else.

*Now* let's calculate the NDVI, and see if that tells us something!  The equation is $NDVI = \frac{nir - red}{nir + red}$, so we'll need to select the `nir` and `red` bands from our array.  Doing arithmetic on arrays in Python operates per-element, so it's really easy when they're the same size!

For example: `array([1, 2, 3]) * array([1, 2, 3]) + array([10, 20, 30])` -> `array([11, 24, 39])`.  
(There are [more complicated rules](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) for arrays of different shapes or dimensions, which you probably don't need to know)

In [ ]:
ndvi = (
      (dat.sel(band="nir") - dat.sel(band="red"))
    / (dat.sel(band="nir") + dat.sel(band="red"))
).rename("NDVI")
ndvi

In [ ]:
ndvi.plot(figsize=(14.8,16))

And it's another diverging colormap - no matter the values of `red` and `nir`, NDVI is always between $\pm$1.  A decent rule of thumb for interpreting NDVI is that negative values are almost always water, small values are dirt or buildings or dry plants, and values above 0.4 or so are live plants.  In this case, Lake Burley Griffin has enough algae in it to have a positive (probably-not-water) NDVI!

Can you see the playing fields in this image?  Let's try plotting again, excluding outliers from the colormap:

In [ ]:
# Is there anything surprising about the fields?  If so, what surprises you and why might it happen?
ndvi.plot(robust=True)

In [ ]:
# Bushfires!

Now that we've found the fake grass, let's turn to a more serious use-case for vegetation indices: assessing the damage and recovery from a bushfire.

This set of images was taken by [Landsat 5](https://en.wikipedia.org/wiki/Landsat_5), the longest-running earth observation satellite to date.  Because this is a well-known and well-calibrated platform, the data has already been processed to give us several advantages during analysis:

- The effect of the atmosphere and angle of the sun have been eliminated to the greatest degree possible
- Instead of brightness, we can work with "reflectance": the proportion of light scattered by the surface.  That should always be between zero and one!
- Every pixel has a known and exact location in time and space, so we can accurately measure distances.

Let's start by opening this new dataset:

In [ ]:
# This data stores each band as a separate variable, like columns in Pandas.
# What advantages might this have?  
ds = xr.open_dataset("bushfire.nc")
ds

- Every dimension has a coordinate index in this dataset.  The coordinates even have units, though they're not shown in this view - you can check `ds.x` or `ds.y` to see that each pixel is 100m across.  The time-index unit is part of the array dtype, datetime with nanosecond precision, rather than Xarray metadata.
- Our data variables include the visible spectrum (blue, green, red), near-infrared, and two shortwave infrared bands.
  The latter are closely related to temperature, and form a crude measurement of it - investigate in free time if you're interested.
- The dataset also has "Attributes", in this case extra metadata about where the image was taken.  Unfortunately there's been a typo in the latitute metadata, but we could in principle recover that from the x-coordinates (geodesy is *way* out of scope today though)

Now let's do a quick sanity-check by calculating `ds.max()` and `ds.min()`.  What do you see?

In [ ]:
# There's an oddity here, which is probably a de-hazing artefact from a smoky image.
# Should we be worried?  You can investigate, and refer to Smith p22, later if you want to.
ds.min()

In [ ]:
# Instead of using `.sel`, we can access the variables directly - like Pandas again!
ds.nir.plot(col="time")

It's already pretty clear that something changed between the first two timesteps!  
Let's take a quick look at *all* the data...

In [ ]:
# This is a relatively expensive operation, because we're creating a whole new array
# with all of that data!  It's not too far from here to "big data" tools...
stacked = xr.concat(
    # The arrays to concatenate, and the name of the new dimension
    [ds.blue, ds.green, ds.red, ds.nir, ds.swir1, ds.swir2], dim="band"
)
# But then we plot it the same way, using `.imshow` for speed.
# (ask Zac for more details you don't need if you're interested)
stacked.plot.imshow(row="time", col="band", robust=True)

Informative, right?  Well... maybe not so much.

- The white patches are 'missing data', usually where the land was covered by clouds.  That can make any band-based analysis nonsensical, so they're usually removed in advance by setting those pixels to the special `NaN` (Not a Number) value.
- There's some spatial structure in the image, which seems related to the change in later time steps.  This is probably a difference in vegetation type, but it's not clear what's what.
- The creek in the top-right is clearly visible in `nir` and `swir1` - we can tell it's a creek by the shape, and characteristic low diffuse reflectivity of water in the infrared spectrum (i.e. it's shiny and cold!)
- Otherwise... ¯\\\_(ツ)\_/¯

This is where you get to write some code: fill out the cell below to create a stacked array with red, green, and blue to see a true-color visualisation of the scene.

In [ ]:
# Hint: you can copy and adjust the `concat` code above.
# If the image doesn't make sense, check the order of the colors!
rgb = xr.concat(
    # The arrays to concatenate, and the name of the new dimension
    [ds.red, ds.green, ds.blue], dim="band"
)
#rgb = rgb.transpose()
# We'll arrange the time axis in left-to-right panels:
rgb.plot.imshow(col="time", robust=True)

Hopefully you can see

- Which areas are dry grass, and which are forest
- The haze and plume of smoke that has *not* been removed, because it wasn't detected as clouds, but will cause similar problems in our analysis.  

*If you want an extension excercise: calculate a boolean "mask" grid which is True for pixels where there is smoke, and False elsewhere.  You can use any combination of bands.  Then create a copy of `ds` with those pixels set to `np.nan`, and do the remaining steps to both.  How do they compare?*

...but it's hard to distinguish dark unburned forest from dark burned forest.  NDVI to the rescue!

In [ ]:
ndvi2 = (
      (ds.nir - ds.red)
    / (ds.nir + ds.red)
).rename("NDVI2")
ndvi2.plot.imshow(col="time", robust=True, figsize=(16,5))
# Plot it here! What happens if you leave out `col="time"`?  Why?

You should be able to see the forest in bright yellow, and grass in shades from dry blue to healthy aqua.
In the second plot facet, the burned area is dark blue - what effect might the smoke have?
In the third facet, some months later, the grass is recovering well due to winter rain but the forested area is still pretty dead.